In [184]:
from bs4 import BeautifulSoup
import requests

In [194]:
url = 'https://www.amazon.com/gp/bestsellers/books' 
response = requests.get(url)
response.status_code

200

In [223]:
page = response.text
soup = BeautifulSoup(page, "lxml")
print(soup.prettify())

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us">
 <!-- sp:feature:head-start -->
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <!-- sp:feature:cs-optimization -->
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
  <link href="https://m.media-amazon.com" rel="dns-prefetch"/>
  <link href="https://completion.amazon.com" rel="dns-prefetch"/>
  <!-- sp:feature:aui-assets -->
  <link href="https://images-na.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|01ZTHTZObnL.css,41SIz69qHYL.css,31qGOnSAToL.css,013z33uKh2L.css,017DsKjNQJL.css,0131vqwP5UL.css,41EWOOlBJ9L.css,11TIuySqr6L.css,01ElnPiDxWL.css,11bGSgD5pDL.css,01Dm5eKVxwL.css,01IdKcBuAdL.css,01y-XAlI+2L.css,21N4kUH7pxL.css,01oDR3IULNL.css,41-PwE7+H0L.css,21j0IlW7xKL.css,01XPHJk60-L.css,014OeDQisGL.css,21aPhFy+riL.css,11gneA3MtJL.css,21fecG8pUzL.css,01RddH8vm-L.css,01CFUgsA-Y

In [224]:
books_list=soup.find_all("li",class_="zg-item-immersion")

In [225]:
books_dict={}
counter=0
for element in books_list:
    
    try:
        book_title=element.find(class_="p13n-sc-truncate p13n-sc-line-clamp-1").text.strip()
    except:
        book_title=""
    
    try:
        book_author=element.find(class_="a-size-small a-link-child").text.strip()
    except:
        book_author=""
        
    try:
        book_price=element.find(class_="p13n-sc-price").text.strip()
    except:
        book_price=""
    
    try:
        book_rating=element.find(class_="a-icon a-icon-star a-star-4-5 aok-align-top").text.strip().split(" ")[0]
    except:
        book_rating=""
        
    try:
        reviews_number=element.find(class_="a-size-small a-link-normal").text.strip()
    except:
        reviews_number=""
        
    
    books_dict[counter]=[book_title,book_author,book_price,book_rating,reviews_number]
    counter+=1

In [226]:
len(books_dict)

50

In [227]:
import pandas as pd

books_df = pd.DataFrame(books_dict).T 
books_df.columns=['Book Name','Book Author',"Book Price","Book Rating","reviews_number"]
books_df.head(5)

,Book Name,Book Author,Book Price,Book Rating,reviews_number
0,It Ends with Us: A Novel,Colleen Hoover,$10.80,4.7,"29,627"
1,Countdown bin Laden: The Untold Story of the 2...,Chris Wallace,$18.84,,50
2,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,$11.98,,"52,068"
3,American Marxism,Mark R. Levin,$16.80,,"12,515"
4,Apples Never Fall,Liane Moriarty,$17.39,,
